# Image Segmentation Task 1
#### Welcome to the first task of Image Segmentation. Image segmentation is the process of partitioning the image into a set of pixels representing an object. In this task, you will be introduced to the problem of image segmentation and programming pipeline involved in image segmentation.

For the purpose of this task we will be using PASCAL VOC datset. The dataset contains a total of 2913 images with segmentation annotations. Code in the cell below will download the code and extract the dataset.

In [1]:
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
#!tar -xvf VOCtrainval_11-May-2012.tar

In [2]:
#!pip install scipy==1.1.0

### 1.1 Loading the dataset

In [3]:
import os
from os.path import join as pjoin
import collections
import json
import torch
import imageio
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt
import glob

from PIL import Image
from tqdm import tqdm
from torch.utils import data
from torchvision import transforms


class pascalVOCDataset(data.Dataset):
    """Data loader for the Pascal VOC semantic segmentation dataset.

    Annotations from both the original VOC data (which consist of RGB images
    in which colours map to specific classes) and the SBD (Berkely) dataset
    (where annotations are stored as .mat files) are converted into a common
    `label_mask` format.  Under this format, each mask is an (M,N) array of
    integer values from 0 to 21, where 0 represents the background class.

    The label masks are stored in a new folder, called `pre_encoded`, which
    is added as a subdirectory of the `SegmentationClass` folder in the
    original Pascal VOC data layout.

    A total of five data splits are provided for working with the VOC data:
        train: The original VOC 2012 training data - 1464 images
        val: The original VOC 2012 validation data - 1449 images
        trainval: The combination of `train` and `val` - 2913 images
        train_aug: The unique images present in both the train split and
                   training images from SBD: - 8829 images (the unique members
                   of the result of combining lists of length 1464 and 8498)
        train_aug_val: The original VOC 2012 validation data minus the images
                   present in `train_aug` (This is done with the same logic as
                   the validation set used in FCN PAMI paper, but with VOC 2012
                   rather than VOC 2011) - 904 images
    """

    def __init__(
        self,
        root,
        sbd_path=None,
        split="train_aug",
        is_transform=False,
        img_size=512,
        augmentations=None,
        img_norm=True,
        test_mode=False,
    ):
        self.root = root
        self.sbd_path = sbd_path
        self.split = split
        self.is_transform = is_transform
        self.augmentations = augmentations
        self.img_norm = img_norm
        self.test_mode = test_mode
        self.n_classes = 21
        self.mean = np.array([104.00699, 116.66877, 122.67892])
        self.files = collections.defaultdict(list)
        self.img_size = img_size if isinstance(img_size, tuple) else (img_size, img_size)

        if not self.test_mode:
            for split in ["train", "val", "trainval"]:
                print(split)
                path = pjoin(self.root, "ImageSets/Segmentation", split + ".txt")
                print(path)
                file_list = tuple(open(path, "r"))
                #print(file_list)
                file_list = [id_.rstrip() for id_ in file_list]
                self.files[split] = file_list
            self.setup_annotations()

        self.tf = transforms.Compose(
            [
                # add more trasnformations as you see fit
                transforms.ToTensor(),
                #transforms.Resize(256),
                #transforms.CenterCrop(256),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        im_name = self.files[self.split][index]
        im_path = pjoin(self.root, "JPEGImages", im_name + ".jpg")
        lbl_path = pjoin(self.root, "SegmentationClass/pre_encoded", im_name + ".png")
        im = Image.open(im_path)
        lbl = Image.open(lbl_path)
        if self.augmentations is not None:
            im, lbl = self.augmentations(im, lbl)
        if self.is_transform:
            im, lbl = self.transform(im, lbl)
        return im, torch.clamp(lbl, max=20)


    def transform(self, img, lbl):
        if self.img_size == ("same", "same"):
            pass
        else:
            img = img.resize((self.img_size[0], self.img_size[1]))  # uint8 with RGB mode
            lbl = lbl.resize((self.img_size[0], self.img_size[1]))
        img = self.tf(img)
        #print(img)
        lbl = torch.from_numpy(np.array(lbl)).long()
        #print(lbl)
        lbl[lbl == 255] = 0
        #print(lbl)
        return img, lbl

    def get_pascal_labels(self):
        """Load the mapping that associates pascal classes with label colors

        Returns:
            np.ndarray with dimensions (21, 3)
        """
        return np.asarray(
            [
                [0, 0, 0],
                [128, 0, 0],
                [0, 128, 0],
                [128, 128, 0],
                [0, 0, 128],
                [128, 0, 128],
                [0, 128, 128],
                [128, 128, 128],
                [64, 0, 0],
                [192, 0, 0],
                [64, 128, 0],
                [192, 128, 0],
                [64, 0, 128],
                [192, 0, 128],
                [64, 128, 128],
                [192, 128, 128],
                [0, 64, 0],
                [128, 64, 0],
                [0, 192, 0],
                [128, 192, 0],
                [0, 64, 128],
            ]
        )

    def encode_segmap(self, mask):
        """Encode segmentation label images as pascal classes

        Args:
            mask (np.ndarray): raw segmentation label image of dimension
              (M, N, 3), in which the Pascal classes are encoded as colours.

        Returns:
            (np.ndarray): class map with dimensions (M,N), where the value at
            a given location is the integer denoting the class index.
        """
        mask = mask.astype(int)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
        for ii, label in enumerate(self.get_pascal_labels()):
            label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
        label_mask = label_mask.astype(int)
        # print(np.unique(label_mask))
        return label_mask

    def decode_segmap(self, label_mask, plot=False):
        """Decode segmentation class labels into a color image

        Args:
            label_mask (np.ndarray): an (M,N) array of integer values denoting
              the class label at each spatial location.
            plot (bool, optional): whether to show the resulting color image
              in a figure.

        Returns:
            (np.ndarray, optional): the resulting decoded color image.
        """
        label_colours = self.get_pascal_labels()
        r = label_mask.copy()
        g = label_mask.copy()
        b = label_mask.copy()
        for ll in range(0, self.n_classes):
            r[label_mask == ll] = label_colours[ll, 0]
            g[label_mask == ll] = label_colours[ll, 1]
            b[label_mask == ll] = label_colours[ll, 2]
        rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
        rgb[:, :, 0] = r / 255.0
        rgb[:, :, 1] = g / 255.0
        rgb[:, :, 2] = b / 255.0
        if plot:
            plt.imshow(rgb)
            plt.show()
        else:
            return rgb

    def setup_annotations(self):
        """Sets up Berkley annotations by adding image indices to the
        `train_aug` split and pre-encode all segmentation labels into the
        common label_mask format (if this has not already been done). This
        function also defines the `train_aug` and `train_aug_val` data splits
        according to the description in the class docstring
        """
        sbd_path = self.sbd_path
        target_path = pjoin(self.root, "SegmentationClass/pre_encoded")
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        train_aug = self.files["train"]

        # keep unique elements (stable)
        train_aug = [train_aug[i] for i in sorted(np.unique(train_aug, return_index=True)[1])]
        self.files["train_aug"] = train_aug
        set_diff = set(self.files["val"]) - set(train_aug)  # remove overlap
        self.files["train_aug_val"] = list(set_diff)

        pre_encoded = glob.glob(pjoin(target_path, "*.png"))
        expected = np.unique(self.files["train_aug"] + self.files["val"]).size

        if len(pre_encoded) != expected:
            print("Pre-encoding segmentation masks...")

            for ii in tqdm(self.files["trainval"]):
                fname = ii + ".png"
                lbl_path = pjoin(self.root, "SegmentationClass", fname)
                lbl = self.encode_segmap(m.imread(lbl_path))
                lbl = m.toimage(lbl, high=lbl.max(), low=lbl.min())
                m.imsave(pjoin(target_path, fname), lbl)

        assert expected == 2913, "unexpected dataset sizes"

### 1.2 Define the model architecture(2.0 point)
In this section you have the freedom to decide your own model. Keep in mind though, to perform image segmentation, you need to implement an architecture that does pixel level classification i.e. for each pixel in the image you need to predict the probability of it belonging to one of the 21 categories.

In [4]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import segmentation as S

#Check if CUDA is available, if not use the CPU. 
train_on_GPU = torch.cuda.is_available()
device = torch.device('cuda' if train_on_GPU else 'cpu')

if train_on_GPU:
    torch.cuda.empty_cache()
    print('CUDA available!')
else:
    print('CUDA not available!')



class Segnet(nn.Module):
    def __init__(self):
        super(Segnet, self).__init__()
        #define the layers for your model

        #Pooling filter with size 2x2, to halve the size of the image
        self.pool = nn.MaxPool2d(2,2)        

        #First VGG layer with two times Convolution and duplicating of the features
        self.conv11 = nn.Conv2d(3,16,3)
        self.conv12 = nn.Conv2d(16,16,3)

        #Second VGG layer with three times Convolution and duplicating of the features
        self.conv21 = nn.Conv2d(16,32,3)
        self.conv22 = nn.Conv2d(32,32,3)
        self.conv23 = nn.Conv2d(32,32,3)

        #Third VGG layer with fully connected layers
        self.fc1 = nn.Linear(32*124*124, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 21)

        resnet50 = S.fcn_resnet50(pretrained=False, progress=True, num_classes=21, aux_loss=None, **kwargs)

        #Output with Softmax because of lossfunction cross entropy
        #self.output = nn.Softmax()

    
    def forward(self, x):
        #print(x)
        print(x.size())
        #define the forward pass
        #output = F.relu(self.pool(x))
        #print(output.size())
        #output = self.out(x)
        #print(output.size())

        #First VGG layer with two times Convolution + ReLu and duplicating of the features
        #Pooling filter with size 2x2, to halve the size of the image, after the second convolution
        print("1. Schicht:")
        out = F.relu(self.conv11(x))
        print(out.size())
        out = self.pool(F.relu(self.conv12(out)))
        print(out.size())

        #Second VGG layer with three times Convolution + ReLu and duplicating of the features
        #Pooling filter with size 2x2, to halve the size of the image, after the third convolution
        print("2. Schicht:")
        out = F.relu(self.conv21(out))
        print(out.size())
        out = F.relu(self.conv22(out))
        print(out.size())
        out = self.pool(F.relu(self.conv23(out)))
        print(out.size())

        #Third VGG layer with fully connected layers
        print("Output Schicht:")
        out = out.view(-1, 32*124*124)
        print(out.size())
        out = F.relu(self.fc1(out))
        print(out.size())
        out = F.relu(self.fc2(out))
        print(out.size())
        out = self.fc3(out)
        print(out)
        print(out.size())
        #out = self.output(out)
        #print(out)

        #We don't use SOFTMAX in the forward process, because we use CrossEntropyLoss() as criterion

        return out
    

CUDA available!


In [5]:
# Creating an instance of the model defined above. 
# You can modify it incase you need to pass paratemers to the constructor.
model = Segnet().to(device)
print(model)

Segnet(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv11): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv12): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv21): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv22): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv23): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=492032, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=21, bias=True)
)


### 1.3 Hyperparameters(0.5 points)
Define all the hyperparameters(not restricted to the three given below) that you find useful here.

In [6]:
local_path = 'VOCdevkit/VOC2012/' # modify it according to your device
bs = 4
epochs = 10
learning_rate = 0.01

### 1.4 Dataset and Dataloader(0.5 points)
Create the dataset using pascalVOCDataset class defined above. Use local_path defined in the cell above as root. 

In [7]:
# dataset variable
dst = pascalVOCDataset(root=local_path,is_transform=True)
print(dst)
# dataloader variable
trainloader = data.DataLoader(dst,batch_size=bs,shuffle=True)
print(trainloader)

train
VOCdevkit/VOC2012/ImageSets/Segmentation\train.txt
val
VOCdevkit/VOC2012/ImageSets/Segmentation\val.txt
trainval
VOCdevkit/VOC2012/ImageSets/Segmentation\trainval.txt


### 1.5 Loss fuction and Optimizer(1.0 point)
Define below with the loss function you think would be most suitable for segmentation task. You are free to choose any optimizer to train the network.

In [8]:
# loss function
loss_f = nn.CrossEntropyLoss()

# optimizer variable
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 1.6 Training the model(3.0 points)
Your task here is to complete the code below to perform a training loop and save the model weights after each epoch of training.

In [9]:
import IPython.display

#if os.path.isfile('saveTask1.pt'):
#    model = torch.load('saveTask1.pt')

#print(os.path.isfile('saveTask1.pt'))

for epo in range(epochs):
    #print(trainloader)
    print(model.parameters)
    for i,data in enumerate(trainloader):
        #"Transport" the data to CUDA if available, and otherwise to the CPU
        image = data[0].to(device)
        label = data[1].to(device)

        #Compute the forward propagation
        output = model(image)

        #Compute the train loss
        print(output, output.size())
        print(label, label.size())
        loss = loss_f(output, label)

        #Reset gradient and execute backpropagation and optimization
        opt.zero_grad()
        loss.backward()
        opt.step()

    print(model.parameters)
    torch.save(model, 'saveTask1.pt')



<bound method Module.parameters of Segnet(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv11): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv12): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv21): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv22): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv23): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=492032, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=21, bias=True)
)>
torch.Size([4, 3, 512, 512])
1. Schicht:
torch.Size([4, 16, 510, 510])
torch.Size([4, 16, 254, 254])
2. Schicht:
torch.Size([4, 32, 252, 252])
torch.Size([4, 32, 250, 250])
torch.Size([4, 32, 124, 124])
Output Schicht:
torch.Size([4, 492032])
torch.Size([4, 64])
torch.Size([4, 16])
tensor([[-0.2613, -0.1340, -0.1623,  0.1002, -0.1705,  0.1965, -0.1512, -0.1558,
          0.

RuntimeError: multi-target not supported at C:/w/b/windows/pytorch/aten/src\THCUNN/generic/ClassNLLCriterion.cu:15

### 1.7 Evaluate your model(1.5 points)
In this section you have to implement the evaluation metrics for your model. Calculate the values of F1-score, dice coefficient and AUC-ROC score on the data you used for training. You can use external packages like scikit-learn to compute above metrics.

In [ ]:
def evaluate(ground_truth, predictions):
    
    return f1_score, auc_score, dice_coeeficient

### 1.8 Plot the evaluation metrics against epochs(1.0)
In section 1.6 we saved the weights of the model after each epoch. In this section, you have to calculate the evaluation metrics after each epoch of training by loading the weights for each epoch. Once you have calculated the evaluation metrics for each epoch, plot them against the epochs.

### 1.9 Visualize results(0.5 points)
For any 10 images in the dataset, show the images along the with their segmentation mask.